# Example of DOV search methods for boringen

@pjhaest
date 20180503

## Use cases:
* Select boreholes in a bbox
* Select boreholes in a bbox with selected properties
* Select boreholes in a municipality
* Select boreholes in a municipality and return depth
* Get boreholes for a municipality where groundwater related data are available



In [57]:
%matplotlib inline
import os, sys
import inspect

In [58]:
# check pydov path
import pydov
print(pydov.__path__)

['c:\\_workingdir\\pydov\\pydov']


## Get information about code base

In [59]:
from pydov.search.boring import BoringSearch
boring = BoringSearch()

# information about the Boring type (In Dutch):
print(boring.get_description())

In de DOV-databank is elke waarneming van grondlagen een boring. Bij de meeste boringen wordt er met een boortoestel een gat gemaakt in de ondergrond om de verschillende grondlagen te kunnen beschrijven. Aan de hand van een boring krijg je een beeld van het materiaal in de ondergrond met toenemende diepte. Afhankelijk van het doel waarvoor de boring geplaatst wordt, zal men een geschikte boormethode toepassen. Boringen worden geplaatst voor verkennend bodemonderzoek, monstername van het sediment en/of grondwater, bepaling van bodemfysische parameters, milieuhygienisch onderzoek,... Afhankelijk van de diepte, soort materiaal, en het al dan niet boren tot onder de grondwatertafel kan men kiezen uit verscheidene systemen voor handmatig of machinaal te boren. Het bodemmateriaal dat vrijkomt, kan gebruikt worden om een profiel van de ondergrond op te stellen of om er grondmonsters van te nemen om verdere analyses op uit te voeren. Vaak is het de bedoeling een put uit te bouwen zodat water k

In [60]:
# information about the available fields for a Boring object
fields = boring.get_fields()
# print available fields
for f in fields.values():
    print(f['name'])

generated_id
id
boornummer
pkey_boring
rapport
diepte_boring_tot
datum_aanvang
namen
putnummer
x
y
start_boring_mtaw
gemeente
uitvoerder
doel
methode
erkenning
opdrachtgever
informele_stratigrafie
formele_stratigrafie
lithologische_beschrijving
gecodeerde_lithologie
hydrogeologische_stratigrafie
quartaire_stratigrafie
geotechnische_codering
informele_hydrostratigrafie
doorheen_quartair
dikte_quartair
tertiair_onder_quartair
opdrachten
mv_mtaw
diepte_boring_van
boorgatmeting
diepte_methode_van
diepte_methode_tot
boormethode


In [61]:
# print information for a certain field
fields['diepte_boring_tot']

{'cost': 1,
 'definition': 'Maximumdiepte van de boring ten opzichte van het aanvangspeil, in meter.',
 'name': 'diepte_boring_tot',
 'notnull': False,
 'type': 'float'}

The cost is an arbitrary attribute to indicate if the information is retrieved from a wfs query (cost = 1), 
or from an xml (cost = 10)

In [62]:
# if an attribute can have several values, these are listed under 'values', e.g. for 'methode':
fields['methode']

{'cost': 1,
 'definition': "De methode waarmee de boring uitgevoerd werd. Heeft als waarde 'onbekend' indien de methode niet gekend is.",
 'name': 'methode',
 'notnull': True,
 'type': 'string',
 'values': ['avegaarboring',
  'droge boring',
  'edelmanboring',
  'geen boring',
  'gestoken boring',
  'graafmachine',
  'handboring',
  'kernboring',
  'lansen',
  'lepelboring',
  'luchthamer',
  'luchthevelboren of air-lift boren',
  'meerdere technieken',
  'omgek. spoelboring',
  'onbekend',
  'pulsboring',
  'ramguts',
  'ramkernboring',
  'rollerbit',
  'slagboring',
  'spade',
  'spiraalboring',
  'spoelboring',
  'steenboring',
  'trilboring',
  'voorput',
  'zuigboring']}

## Try-out of use cases

### Select boreholes in a bbox

In [63]:
# Get all borehole data in a bounding box (llx, llxy, ulx, uly)
# the pkey_boring link is not available below, but is in the df
df = boring.search(location=(151650, 214675, 151750, 214775))
print(df.head())

                                         pkey_boring         boornummer  \
0  https://www.dov.vlaanderen.be/data/boring/2004...  GEO-04/169-BNo-B1   
1  https://www.dov.vlaanderen.be/data/boring/2004...  GEO-04/169-BNo-B1   

           x          y mv_mtaw  start_boring_mtaw   gemeente  \
0  151680.75  214678.06    None               7.16  Antwerpen   
1  151680.75  214678.06    None               7.16  Antwerpen   

   diepte_boring_van  diepte_boring_tot datum_aanvang uitvoerder  \
0                0.0               30.0    2004-12-20     GEOLAB   
1                0.0               30.0    2004-12-20     GEOLAB   

   boorgatmeting  diepte_methode_van  diepte_methode_tot    boormethode  
0          False                 0.0                 1.5          spade  
1          False                 1.5                30.0  avegaarboring  


The dataframe contains one borehole where two methodes ('boormethode') were applied for its construction. The available data are flattened to represent unique attributes per row of the dataframe

### Select boreholes in a bbox with selected properties

In [64]:
# list available query methods
methods = [i for i,j in inspect.getmembers(sys.modules['owslib.fes'], 
                                           inspect.isclass) 
           if 'Property' in i]
print(methods)
from owslib.fes import PropertyIsGreaterThanOrEqualTo

['PropertyIsBetween', 'PropertyIsEqualTo', 'PropertyIsGreaterThan', 'PropertyIsGreaterThanOrEqualTo', 'PropertyIsLessThan', 'PropertyIsLessThanOrEqualTo', 'PropertyIsLike', 'PropertyIsNotEqualTo', 'PropertyIsNull', 'SortProperty']


The property feature methodes listed above are available from the owslib module. These were not adapted for use in pydov.

In [65]:
# Get deep boreholes in a bounding box 
from owslib.fes import PropertyIsEqualTo
# the propertyname can be any of the fields of the boring object as shown on line 52 above, the literal is always a string, no 
# matter what its definition is in the boring object (string, float...)
query = PropertyIsGreaterThanOrEqualTo(
        propertyname='diepte_boring_tot', literal='2000')
df = boring.search(
    location=(200000, 211000, 205000, 214000),
    query=query
    )
print(df.head())

                                         pkey_boring  boornummer         x  \
0  https://www.dov.vlaanderen.be/data/boring/2016...  B/1-102785  201790.5   
1  https://www.dov.vlaanderen.be/data/boring/2016...  B/1-102786  201783.0   
2  https://www.dov.vlaanderen.be/data/boring/2016...  B/1-102786  201783.0   
3  https://www.dov.vlaanderen.be/data/boring/2016...  B/1-102786  201783.0   
4  https://www.dov.vlaanderen.be/data/boring/2016...  B/1-102786  201783.0   

          y  mv_mtaw  start_boring_mtaw gemeente  diepte_boring_van  \
0  212962.0     25.0               25.0      Mol                0.0   
1  212961.0     25.0               25.0      Mol                0.0   
2  212961.0     25.0               25.0      Mol                0.0   
3  212961.0     25.0               25.0      Mol                0.0   
4  212961.0     25.0               25.0      Mol                0.0   

   diepte_boring_tot datum_aanvang          uitvoerder  boorgatmeting  \
0             4341.0    2016-03

### Select boreholes in a municipality

In [66]:
query = PropertyIsEqualTo(propertyname='gemeente',
                          literal='Herstappe')
df = boring.search(query=query)
print(df.head())

                                         pkey_boring       boornummer  \
0  https://www.dov.vlaanderen.be/data/boring/2016...   kb33d106e-B236   
1  https://www.dov.vlaanderen.be/data/boring/1993...  kb41d120e-B1027   

          x         y  mv_mtaw  start_boring_mtaw   gemeente  \
0  224687.4  158191.0    124.0              124.0  Herstappe   
1  224820.0  157794.0    132.0              132.0  Herstappe   

   diepte_boring_van  diepte_boring_tot datum_aanvang  \
0                0.0                6.0          None   
1                0.0               50.0    1993-09-02   

                           uitvoerder  boorgatmeting  diepte_methode_van  \
0  Belgische Geologische Dienst (BGD)          False                 0.0   
1                      Peeters-Ramsel          False                 0.0   

   diepte_methode_tot      boormethode  
0                 6.0  gestoken boring  
1                50.0         onbekend  


### Select boreholes in a municipality and return depth

In [67]:
query = PropertyIsEqualTo(propertyname='gemeente',
                          literal='Gent')
df = boring.search(query=query,
                   return_fields=('diepte_boring_tot',))
print(df.head())
# get general info of dataframe
print(df.describe())
# depth of borehole equal to zero indicates missing values 
# a depth could be available in the interpretations

   diepte_boring_tot
0               42.0
1               32.0
2               59.0
3               11.0
4               27.0
       diepte_boring_tot
count        2628.000000
mean           15.534992
std            32.308829
min             0.000000
25%             0.000000
50%             6.100000
75%            20.000000
max           660.000000


### Get boreholes for a municipality where groundwater related data are available

In [68]:
#  E.g. get the boreholes where 'gemeente' is 'Antwerp' and either 
# 'putnummer' is not empty 
# or 'doel' starts with 'Grondwater' 
# or 'erkenning' equals '2. Andere grondwaterwinningen'
from owslib.fes import PropertyIsLike
from owslib.fes import PropertyIsNull
from owslib.fes import And
from owslib.fes import Or
from owslib.fes import Not
query = And([PropertyIsEqualTo(propertyname='gemeente',
                               literal='Antwerpen'),
             Or([Not([PropertyIsNull(propertyname='putnummer')]),
                PropertyIsLike(propertyname='doel',
                               literal='Grondwater%'),
                PropertyIsEqualTo(propertyname='erkenning', 
                                  literal='2. Andere grondwaterwinningen')]
               )]
           )
df = boring.search(query=query)
print(df.head())


                                         pkey_boring       boornummer  \
0  https://www.dov.vlaanderen.be/data/boring/1997...    kb15d28e-B741   
1  https://www.dov.vlaanderen.be/data/boring/1996...    kb15d43w-B393   
2  https://www.dov.vlaanderen.be/data/boring/2016...       B/1-102776   
3  https://www.dov.vlaanderen.be/data/boring/2017...       B/1-102987   
4  https://www.dov.vlaanderen.be/data/boring/2017...  1434-BG000106-9   

          x         y  mv_mtaw  start_boring_mtaw   gemeente  \
0  154825.0  212445.0     5.00               5.00  Antwerpen   
1  151150.0  206550.0    18.09              18.09  Antwerpen   
2  153099.0  213577.0    -1.00              -1.00  Antwerpen   
3  149408.0  224256.0    -1.00              -1.00  Antwerpen   
4  149486.0  221697.0     5.98               5.98  Antwerpen   

   diepte_boring_van  diepte_boring_tot datum_aanvang  \
0                0.0               36.5    1997-08-21   
1                0.0              104.0    1996-06-26   
2    